# Imports
- Devido à alguns erros ao tentar instalar o pacote **basedosdados**, acabei optando por instalar uma versão um pouco mais antiga do pacote
<br>pip install basedosdados==2.0.0b15
- Para que ela funcione ainda precisei atualizar os pacotes numpy e pandas
<br>pip install --upgrade pandas
<br>pip install --upgrade numpy

**API Reference:** https://basedosdados.github.io/mais/api_reference_python/

In [12]:
import basedosdados as bd

# Data

In [20]:
# BigQuery Script
query_b = "SELECT * FROM datario.dados_mestres.bairro LIMIT 10"
query_c = "SELECT * FROM datario.adm_central_atendimento_1746.chamado LIMIT 10"
query_e = "SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos LIMIT 10"

# Retrieve data
bairro = bd.read_sql(query_b, billing_project_id="dados-rio-433111")
chamado = bd.read_sql(query_c, billing_project_id="dados-rio-433111")
eventos = bd.read_sql(query_e, billing_project_id="dados-rio-433111")

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [39]:
# Foi observado que o método read_sql já retorna um objeto DataFrame Pandas, e os métodos e atributos de dataframe pandas podem ser utilizados sem a necessidade de importar a biblioteca 

# Quantidade e nomes das colunas de cada dataset a ser trabalhado no desafio
datasets = {'bairro':bairro, 'chamado':chamado, 'eventos':eventos}
for ds,data in datasets.items():
    print(f'Dataset: {ds}\nQT_Columns: {data.shape[1]}\nColumns: {data.columns}\n')

Dataset: bairro
QT_Columns: 12
Columns: Index(['id_bairro', 'nome', 'id_area_planejamento', 'id_regiao_planejamento',
       'nome_regiao_planejamento', 'id_regiao_administrativa',
       'nome_regiao_administrativa', 'subprefeitura', 'area', 'perimetro',
       'geometry_wkt', 'geometry'],
      dtype='object')

Dataset: chamado
QT_Columns: 32
Columns: Index(['id_chamado', 'id_origem_ocorrencia', 'data_inicio', 'data_fim',
       'id_bairro', 'id_territorialidade', 'id_logradouro',
       'numero_logradouro', 'id_unidade_organizacional',
       'nome_unidade_organizacional', 'id_unidade_organizacional_mae',
       'unidade_organizacional_ouvidoria', 'categoria', 'id_tipo', 'tipo',
       'id_subtipo', 'subtipo', 'status', 'longitude', 'latitude',
       'data_alvo_finalizacao', 'data_alvo_diagnostico',
       'data_real_diagnostico', 'tempo_prazo', 'prazo_unidade', 'prazo_tipo',
       'dentro_prazo', 'situacao', 'tipo_situacao', 'justificativa_status',
       'reclamacoes', 'data_par

# Exploring the APIs

## Public Holidays API

- https://date.nager.at/swagger/index.html
- https://github.com/nager/Nager.Date
- https://date.nager.at/Api

In [81]:
import json
import requests
import pandas as pd

response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)

In [75]:
holidays = pd.DataFrame(public_holidays)
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         14 non-null     object
 1   localName    14 non-null     object
 2   name         14 non-null     object
 3   countryCode  14 non-null     object
 4   fixed        14 non-null     bool  
 5   global       14 non-null     bool  
 6   counties     1 non-null      object
 7   launchYear   0 non-null      object
 8   types        14 non-null     object
dtypes: bool(2), object(7)
memory usage: 944.0+ bytes


In [82]:
holidays

,date,localName,name,countryCode,fixed,global,counties,launchYear,types
0,2024-01-01,Confraternização Universal,New Year's Day,BR,False,True,None,None,[Public]
1,2024-02-12,Carnaval,Carnival,BR,False,True,None,None,"[Bank, Optional]"
2,2024-02-13,Carnaval,Carnival,BR,False,True,None,None,"[Bank, Optional]"
3,2024-03-29,Sexta-feira Santa,Good Friday,BR,False,True,None,None,[Public]
4,2024-03-31,Domingo de Páscoa,Easter Sunday,BR,False,True,None,None,[Public]
5,2024-04-21,Dia de Tiradentes,Tiradentes,BR,False,True,None,None,[Public]
6,2024-05-01,Dia do Trabalhador,Labour Day,BR,False,True,None,None,[Public]
7,2024-05-30,Corpus Christi,Corpus Christi,BR,False,True,None,None,[Public]
8,2024-07-09,Revolução Constitucionalista de 1932,Constitutionalist Revolution of 1932,BR,False,False,[BR-SP],None,[Public]
9,2024-09-07,Dia da Independência,Independence Day,BR,False,True,None,None,[Public]


## Open-Meteo Historical Weather API

- De acordo com http://www.rio.rj.gov.br, a cidade do Rio de Janeiro está situada a 22º54'23" de latitude sul e 43º10'21" de longitude oeste, no município do mesmo nome: é a capital do Estado do Rio de Janeiro, um dos componentes da Região Sudeste do Brasil.
<br>lat: 22.5423
<br>lon: 43.1021